In [1]:
# Dependencies and Setup
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import csv
import json
import calendar
from datetime import date
import random

In [2]:
wardHHIncome = pd.read_csv("Resources/Ward_HH_Income.csv").set_index("WardNumber")

In [3]:
def ward_data(latitude, longitude):
    try:
        ward_url = f"https://represent.opennorth.ca/boundaries/?contains={latitude},{longitude}"
        to_wards = requests.get(ward_url).json()
        ward_data = [[f"Ward {to_wards['objects'][i]['external_id']}",to_wards["objects"][i]["name"].replace(f"({to_wards['objects'][i]['external_id']})",""),wardHHIncome.loc[int(to_wards['objects'][i]['external_id'])]["MedianHHIncome"]] for i in range(len(to_wards["objects"])) if (to_wards["objects"][i]["boundary_set_name"] == "Toronto ward") if (to_wards["objects"][i]["related"]["boundary_set_url"] == "/boundary-sets/toronto-wards-2010/")] 
        if len(ward_data) == 0:
            ward_data = [[f"Ward {to_wards['objects'][i]['external_id']}",to_wards["objects"][i]["name"].replace(f"({to_wards['objects'][i]['external_id']})",""),wardHHIncome.loc[int(to_wards['objects'][i]['external_id'])]["MedianHHIncome"]] for i in range(len(to_wards["objects"])) if (to_wards["objects"][i]["boundary_set_name"] == "Toronto ward") if (to_wards["objects"][i]["related"]["boundary_set_url"] == "/boundary-sets/toronto-wards-2018/")] 

        #print(f"Ward number: {ward_data[0][0]} | Ward name: {ward_data[0][1]} | Median HH Income: {ward_data[0][2]}" )
    except Exception as e:
        print(f"Failed to find boundaries for latitude {latitude} and longitude {longitude}")
        ward_data = 0
    return ward_data

In [4]:
#Load full dataframe
mci_full_df = pd.read_csv("Resources/MCI_2014_to_2018.csv").sort_values(["Lat", "Long"])

# Add empty columns for wardname and wardmedian
#mci_full_df["wardname"]=""
#mci_full_df["wardmedian"]=0
mci_full_df["Lat"] = mci_full_df["Lat"].astype(float)
mci_full_df["Long"] = mci_full_df["Long"].astype(float)

In [18]:
lat_long_df = mci_full_df[["Lat","Long"]].sort_values(["Lat", "Long"])

In [19]:
lat_long_df = lat_long_df.drop_duplicates()

In [8]:
columns = ["Lat", "Long", "wardnumber", "wardname", "wardmedian"]
ward_lat_long_df = pd.DataFrame(columns=columns)

In [27]:
i = 0 
for coordinate in lat_long_df.itertuples():
    j=0
    while True:
        try:
            warddata = ward_data(coordinate.Lat, coordinate.Long)
            print(f"index: {i} | Latitude: {coordinate.Lat} | Longitude: {coordinate.Long} | {warddata[0][1]}")
            
            lat_long_l = list([coordinate.Lat, coordinate.Long,warddata[0][0],warddata[0][1],warddata[0][2]])
            ward_lat_long_df = pd.concat([ward_lat_long_df,pd.DataFrame([lat_long_l], columns=columns)])
            i = i + 1
            break
        except Exception as e:
            print(f"Re-try number: {j}\n{e}")
            print(warddata)
            j = j + 1
            if j > 100:
                break
            time.sleep(random.randint(0,9))


Re-try number: 0
list index out of range
[]
Re-try number: 1
list index out of range
[]
index: 0 | Latitude: 43.7498817 | Longitude: -79.23146059999999 | Scarborough Centre 
index: 1 | Latitude: 43.749885600000006 | Longitude: -79.6020813 | Etobicoke North 
index: 2 | Latitude: 43.7498932 | Longitude: -79.2730255 | Scarborough Centre 
index: 3 | Latitude: 43.7499008 | Longitude: -79.33779140000001 | Don Valley East 
index: 4 | Latitude: 43.7499313 | Longitude: -79.4412766 | York Centre 
index: 5 | Latitude: 43.7499466 | Longitude: -79.55231479999999 | York West 
index: 6 | Latitude: 43.749954200000005 | Longitude: -79.43716429999999 | York Centre 
index: 7 | Latitude: 43.749954200000005 | Longitude: -79.3073502 | Scarborough Centre 
index: 8 | Latitude: 43.749954200000005 | Longitude: -79.2917938 | Scarborough Centre 
index: 9 | Latitude: 43.749958 | Longitude: -79.39589690000001 | Don Valley West 
index: 10 | Latitude: 43.7499924 | Longitude: -79.49077609999999 | York Centre 
index: 1

index: 101 | Latitude: 43.750915500000005 | Longitude: -79.24230190000002 | Scarborough Centre 
index: 102 | Latitude: 43.7509193 | Longitude: -79.3156357 | Don Valley East 
index: 103 | Latitude: 43.7509193 | Longitude: -79.2403793 | Scarborough Centre 
index: 104 | Latitude: 43.750927000000004 | Longitude: -79.18768309999999 | Scarborough East 
index: 105 | Latitude: 43.7509346 | Longitude: -79.4038315 | Don Valley West 
index: 106 | Latitude: 43.7509346 | Longitude: -79.39829250000001 | Don Valley West 
index: 107 | Latitude: 43.750942200000004 | Longitude: -79.30107120000001 | Scarborough Centre 
index: 108 | Latitude: 43.7509499 | Longitude: -79.59415440000001 | Etobicoke North 
index: 109 | Latitude: 43.7509499 | Longitude: -79.444519 | York Centre 
index: 110 | Latitude: 43.7509575 | Longitude: -79.36150359999999 | Don Valley West 
index: 111 | Latitude: 43.75096129999999 | Longitude: -79.5411224 | York West 
index: 112 | Latitude: 43.7509766 | Longitude: -79.5858307 | Etobicoke

index: 201 | Latitude: 43.7519531 | Longitude: -79.4449463 | York Centre 
index: 202 | Latitude: 43.75196079999999 | Longitude: -79.2468872 | Scarborough Centre 
index: 203 | Latitude: 43.7519646 | Longitude: -79.2490005 | Scarborough Centre 
index: 204 | Latitude: 43.7519875 | Longitude: -79.3337021 | Don Valley East 
index: 205 | Latitude: 43.752018 | Longitude: -79.5959244 | Etobicoke North 
index: 206 | Latitude: 43.752025599999996 | Longitude: -79.4881744 | York Centre 
index: 207 | Latitude: 43.75203320000001 | Longitude: -79.43939209999999 | York Centre 
index: 208 | Latitude: 43.752037 | Longitude: -79.29744720000001 | Scarborough Centre 
index: 209 | Latitude: 43.752044700000006 | Longitude: -79.38163759999999 | Don Valley West 
index: 210 | Latitude: 43.752067600000004 | Longitude: -79.2624588 | Scarborough Centre 
index: 211 | Latitude: 43.7520905 | Longitude: -79.2444153 | Scarborough Centre 
index: 212 | Latitude: 43.752094299999996 | Longitude: -79.5423813 | York West 
in

index: 302 | Latitude: 43.7531586 | Longitude: -79.33908840000001 | Don Valley East 
index: 303 | Latitude: 43.7531662 | Longitude: -79.40522 | Don Valley West 
index: 304 | Latitude: 43.753170000000004 | Longitude: -79.3403244 | Don Valley East 
index: 305 | Latitude: 43.753181500000004 | Longitude: -79.4199753 | Willowdale 
index: 306 | Latitude: 43.7531891 | Longitude: -79.415741 | Willowdale 
index: 307 | Latitude: 43.7531929 | Longitude: -79.2477875 | Scarborough Centre 
index: 308 | Latitude: 43.7532082 | Longitude: -79.17977909999999 | Scarborough East 
index: 309 | Latitude: 43.753212 | Longitude: -79.197937 | Scarborough East 
index: 310 | Latitude: 43.7532158 | Longitude: -79.2511444 | Scarborough Centre 
index: 311 | Latitude: 43.753231 | Longitude: -79.2762833 | Scarborough Centre 
index: 312 | Latitude: 43.753234899999995 | Longitude: -79.3944702 | Don Valley West 
index: 313 | Latitude: 43.7532539 | Longitude: -79.265152 | Scarborough Centre 
index: 314 | Latitude: 43.753

index: 405 | Latitude: 43.7544403 | Longitude: -79.5309067 | York West 
index: 406 | Latitude: 43.75445560000001 | Longitude: -79.31251529999999 | Scarborough Centre 
index: 407 | Latitude: 43.7544708 | Longitude: -79.5307541 | York West 
index: 408 | Latitude: 43.7544785 | Longitude: -79.3840179 | Don Valley West 
index: 409 | Latitude: 43.7544861 | Longitude: -79.51812740000001 | York West 
index: 410 | Latitude: 43.754493700000005 | Longitude: -79.46372990000002 | York Centre 
index: 411 | Latitude: 43.7544975 | Longitude: -79.4382401 | York Centre 
index: 412 | Latitude: 43.754505200000004 | Longitude: -79.4434662 | York Centre 
index: 413 | Latitude: 43.7545166 | Longitude: -79.4887619 | York West 
index: 414 | Latitude: 43.7545204 | Longitude: -79.4840622 | York West 
index: 415 | Latitude: 43.7545662 | Longitude: -79.5659561 | York West 
index: 416 | Latitude: 43.7545662 | Longitude: -79.45350649999999 | York Centre 
index: 417 | Latitude: 43.75457 | Longitude: -79.443161 | York

...

index: 4836 | Latitude: 43.8149033 | Longitude: -79.2251282 | Scarborough-Rouge River 
index: 4837 | Latitude: 43.8149338 | Longitude: -79.2870178 | Scarborough-Rouge River 
index: 4838 | Latitude: 43.8149338 | Longitude: -79.22166440000001 | Scarborough-Rouge River 
index: 4839 | Latitude: 43.8150024 | Longitude: -79.31320190000001 | Scarborough-Agincourt 
index: 4840 | Latitude: 43.8150749 | Longitude: -79.3273087 | Scarborough-Agincourt 
index: 4841 | Latitude: 43.8152542 | Longitude: -79.2323532 | Scarborough-Rouge River 
index: 4842 | Latitude: 43.815261799999995 | Longitude: -79.21865079999999 | Scarborough-Rouge River 
index: 4843 | Latitude: 43.8152809 | Longitude: -79.202652 | Scarborough-Rouge River 
index: 4844 | Latitude: 43.8152885 | Longitude: -79.2234955 | Scarborough-Rouge River 
index: 4845 | Latitude: 43.815360999999996 | Longitude: -79.2640228 | Scarborough-Rouge River 
index: 4846 | Latitude: 43.8154297 | Longitude: -79.2314987 | Scarborough-Rouge River 
index: 4847

index: 4928 | Latitude: 43.8181229 | Longitude: -79.2938309 | Scarborough-Rouge River 
index: 4929 | Latitude: 43.818149600000005 | Longitude: -79.2809677 | Scarborough-Rouge River 
index: 4930 | Latitude: 43.8181534 | Longitude: -79.2049179 | Scarborough-Rouge River 
index: 4931 | Latitude: 43.8181648 | Longitude: -79.31761170000001 | Scarborough-Agincourt 
index: 4932 | Latitude: 43.818172499999996 | Longitude: -79.281723 | Scarborough-Rouge River 
index: 4933 | Latitude: 43.8182297 | Longitude: -79.3057175 | Scarborough-Agincourt 
index: 4934 | Latitude: 43.818248700000005 | Longitude: -79.1757202 | Scarborough-Rouge River 
index: 4935 | Latitude: 43.8183937 | Longitude: -79.279953 | Scarborough-Rouge River 
index: 4936 | Latitude: 43.818420399999994 | Longitude: -79.23985290000002 | Scarborough-Rouge River 
index: 4937 | Latitude: 43.8184357 | Longitude: -79.2910233 | Scarborough-Rouge River 
index: 4938 | Latitude: 43.8184433 | Longitude: -79.2119064 | Scarborough-Rouge River 
ind

index: 5019 | Latitude: 43.8211174 | Longitude: -79.2852402 | Scarborough-Rouge River 
index: 5020 | Latitude: 43.82113270000001 | Longitude: -79.2262421 | Scarborough-Rouge River 
index: 5021 | Latitude: 43.821209 | Longitude: -79.27085109999999 | Scarborough-Rouge River 
index: 5022 | Latitude: 43.8212776 | Longitude: -79.27589420000001 | Scarborough-Rouge River 
index: 5023 | Latitude: 43.8213005 | Longitude: -79.2051392 | Scarborough-Rouge River 
index: 5024 | Latitude: 43.821304299999994 | Longitude: -79.2843628 | Scarborough-Rouge River 
index: 5025 | Latitude: 43.8213425 | Longitude: -79.2250977 | Scarborough-Rouge River 
index: 5026 | Latitude: 43.8213463 | Longitude: -79.2829437 | Scarborough-Rouge River 
index: 5027 | Latitude: 43.821373 | Longitude: -79.22036740000001 | Scarborough-Rouge River 
index: 5028 | Latitude: 43.8213959 | Longitude: -79.316925 | Scarborough-Agincourt 
index: 5029 | Latitude: 43.8214722 | Longitude: -79.27307890000002 | Scarborough-Rouge River 
index

index: 5109 | Latitude: 43.8249817 | Longitude: -79.30152890000001 | Scarborough-Rouge River 
index: 5110 | Latitude: 43.8249817 | Longitude: -79.2662354 | Scarborough-Rouge River 
index: 5111 | Latitude: 43.8249855 | Longitude: -79.2906876 | Scarborough-Rouge River 
index: 5112 | Latitude: 43.8249855 | Longitude: -79.21893309999999 | Scarborough-Rouge River 
index: 5113 | Latitude: 43.8249893 | Longitude: -79.2860565 | Scarborough-Rouge River 
index: 5114 | Latitude: 43.8249893 | Longitude: -79.2725143 | Scarborough-Rouge River 
index: 5115 | Latitude: 43.8251419 | Longitude: -79.2253189 | Scarborough-Rouge River 
index: 5116 | Latitude: 43.8251534 | Longitude: -79.18875890000001 | Scarborough-Rouge River 
index: 5117 | Latitude: 43.8251686 | Longitude: -79.2961578 | Scarborough-Rouge River 
index: 5118 | Latitude: 43.825180100000004 | Longitude: -79.2235031 | Scarborough-Rouge River 
index: 5119 | Latitude: 43.825199100000006 | Longitude: -79.22026059999999 | Scarborough-Rouge River 

index: 5200 | Latitude: 43.832649200000006 | Longitude: -79.22742459999999 | Scarborough-Rouge River 
index: 5201 | Latitude: 43.8327065 | Longitude: -79.26795959999998 | Scarborough-Rouge River 
index: 5202 | Latitude: 43.832977299999996 | Longitude: -79.22485350000001 | Scarborough-Rouge River 
index: 5203 | Latitude: 43.8333588 | Longitude: -79.2272263 | Scarborough-Rouge River 
index: 5204 | Latitude: 43.833454100000004 | Longitude: -79.2510223 | Scarborough-Rouge River 
index: 5205 | Latitude: 43.8335533 | Longitude: -79.22532650000001 | Scarborough-Rouge River 
index: 5206 | Latitude: 43.8336105 | Longitude: -79.23043059999999 | Scarborough-Rouge River 
index: 5207 | Latitude: 43.8340263 | Longitude: -79.2270432 | Scarborough-Rouge River 
index: 5208 | Latitude: 43.834243799999996 | Longitude: -79.2301865 | Scarborough-Rouge River 
index: 5209 | Latitude: 43.8342896 | Longitude: -79.2275696 | Scarborough-Rouge River 
index: 5210 | Latitude: 43.8343048 | Longitude: -79.26115420000

In [34]:
ward_lat_long_df.count()


Lat           17330
Long          17330
wardnumber    17330
wardname      17330
wardmedian    17330
dtype: int64

In [29]:
ward_lat_long_df["wardmedian"] = ward_lat_long_df["wardmedian"].astype(float)
ward_lat_long_df = ward_lat_long_df.sort_values(["Lat","Long"])

In [ ]:
#lat_long_df = ward_lat_long_df.loc[ward_lat_long_df["wardmedian"]==0][["Lat","Long"]]

In [ ]:
#lat_long_df = lat_long_df.drop_duplicates()

In [ ]:
#lat_long_df.head(10)

In [ ]:
#ward_lat_long_df = ward_lat_long_df.sort_values(["wardmedian"])

In [ ]:
#ward_lat_long_df

In [ ]:
#ward_lat_long_df = ward_lat_long_df.loc[ward_lat_long_df['wardmedian']!=0]

In [ ]:
#ward_lat_long_df = ward_lat_long_df.drop_duplicates()

In [ ]:
#ward_lat_long_df = ward_lat_long_df.loc[ward_lat_long_df['wardmedian']!=0]

In [ ]:
#ward_lat_long_df

In [ ]:
#ward_lat_long_df = ward_lat_long_df.drop_duplicates()

In [ ]:
#ward_lat_long_df.loc[ward_lat_long_df['wardmedian']!=0].count()

In [30]:
mci_full_df1 = pd.merge(mci_full_df,ward_lat_long_df,how='inner',on=["Lat","Long"])

In [31]:
mci_full_df1 = mci_full_df1.drop_duplicates()

In [33]:
mci_full_df1.to_csv("Resources/MCI_2014_to_2018_full.csv")

In [32]:
mci_full_df.count()

X                      167525
Y                      167525
Index_                 167525
event_unique_id        167525
occurrencedate         167525
reporteddate           167525
premisetype            167525
ucr_code               167525
ucr_ext                167525
offence                167525
reportedyear           167525
reportedmonth          167525
reportedday            167525
reporteddayofyear      167525
reporteddayofweek      167525
reportedhour           167525
occurrenceyear         167476
occurrencemonth        167476
occurrenceday          167476
occurrencedayofyear    167476
occurrencedayofweek    167476
occurrencehour         167525
MCI                    167525
Division               167525
Hood_ID                167525
Neighbourhood          167525
Lat                    167525
Long                   167525
ObjectId               167525
dtype: int64